# Mine Sweeperを作る
行儀の良いコードを作るのにすこし飽きてきたので、ゲームをひとつ作ってみます。

マインスイーパーはこんなゲームです。→http://www.afsgames.com/mine.htm

ゲームのルールを修得する意味で、すこし自分で遊んでみて下さい。

今回は、コンピュータが作った問題を人間が解くことにします。(コンピュータに解かせるのは格段に難しいですが、人間の考えをアルゴリズム化すればいいので、不可能ではありません)

まだグラフィックスの使い方を学んでいない(というより、WinPythonはあまりそういう用途を想定していない)ので、テキスト表示の範囲で作ります。当然、座標を直接マウスでは指定できないので、input命令でx,y座標を入れさせます。

## 初級: 紙ベースのマインスイーパー
まずは、一番原始的なマインスイーパーを作ります。

1. コンピュータが盤を準備する。
2. 人間が、座標を指定する。
3. コンピュータは、そこが爆弾か、あるいは周囲8格子点の爆弾の数を答える。
4. 爆弾にヒットするまで2,3を繰り返す。

### 盤を作る
盤の大きさは、とりあえず10x10としましょう。また、爆弾の総数は10個とします。

どの場所に爆弾があるか、という情報をPythonに覚えさせる方法はいくつかあります。リスト、二次元リスト、集合、辞書のどれでも書けますが、今回は集合を使うことにします。

爆弾の座標の集合を作ります。randomライブラリのrandint関数で、0〜9の整数をランダムに発生させられます。

In [ ]:
import random
nbomb = 10
mapsize = (10,10)

bombs = set()
for i in range(nbomb):
    while True:                 #無条件ループ
        x = random.randint(0,9)
        y = random.randint(0,9)
        if (x,y) not in bombs:
            break               #whileループから抜ける
    bombs.add((x,y))
print(bombs)

ちゃんと動くことがわかったら、さっそく関数にしてしまいます。

In [ ]:
import random

def makemap(mapsize,nbomb):
    bombs = set()
    for i in range(nbomb):
        while True:                 #無条件ループ
            x = random.randint(0,mapsize[0]-1)
            y = random.randint(0,mapsize[1]-1)
            if (x,y) not in bombs:
                break               #whileループから抜ける
        bombs.add((x,y))
    return bombs

nbomb = 10
mapsize = (10,10)
bombs = makemap(mapsize,nbomb)
print(bombs)

入力は、input命令を使うことにします。数字の区切りにはcommaが必要です。

In [ ]:
x,y=eval(input("X Y:"))

もしその座標に爆弾があれば、もうおしまいです。

In [ ]:
if (x,y) in bombs:
    print("BOMB!")

そうでない場合には、周囲8格子点にある爆弾の総数を表示します。

In [ ]:
else:
    bombcount = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                bombcount += 1
    print("There are {0} bombs near ({1},{2}).".format(bombcount,x,y))

ある場所の周囲9点の爆弾の個数を数える部分も関数countbombs()にしてしまいましょう。

In [ ]:
def countbombs(x,y,bombs):
    n = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                n += 1
    return n

あとは、入力をずっと繰りかえすだけです。全部を組みあわせると、次のようになります。

In [ ]:
import random



def countbombs(x,y,bombs):
    n = 0
    for neix in range(x-1,x+2):
        for neiy in range(y-1,y+2):
            if (neix,neiy) in bombs:
                n += 1
    return n



def makemap(mapsize,nbomb):
    bombs = set()
    for i in range(nbomb):
        while True:                 #無条件ループ
            x = random.randint(0,mapsize[0]-1)
            y = random.randint(0,mapsize[1]-1)
            if (x,y) not in bombs:
                break               #whileループから抜ける
        bombs.add((x,y))
    return bombs

nbomb = 10
mapsize = (10,10)
bombs = makemap(mapsize,nbomb)

while True:                         #また無条件ループ
    x,y=eval(input("X Y:"))
    if (x,y) in bombs:
        print("BOMB!")
        break                       #whileループから抜ける
    else:
        bombcount = countbombs(x,y,bombs)
        print("There are {0} bombs near ({1},{2}).".format(bombcount,x,y))


## 中級編: 画面表示
紙ベースのマインスイーパーは、どこをチェックしたか、という情報はぜんぶ自分で紙(方眼紙)に書き残しておかなければならず、けっこう面倒です。そこで、すでにチェックした場所が地図の形で表示されるようにしてみましょう。そのためには、プレーヤーがこれまで入力した座標と、その周辺の爆弾の個数を全部記憶しておく必要があります。これには辞書を使うのがぴったりです。

座標をキーとし、近傍の爆弾の個数を値とする辞書numを準備します。

In [ ]:
num = dict()

そして、爆弾の数を毎度保管していきます。

In [ ]:
num[(x,y)] = bombcount

### 課題1
この2行を、上のプログラムの適切な場所に加えて下さい。

マインスイーパーで表示されるマップは、numの中身です。numを10x10の格子の形で表示することにしましょう。安全かどうか確認できていない場所には"?"と表示することにします。

In [ ]:
def showmap(num,mapsize):
    for posy in range(mapsize[1]):
        for posx in range(mapsize[0]):
            if (posx,posy) in num:
                bombcount = num[(posx,posy)]
                print(bombcount,end="")
            else:
                print("?",end="")
        print()

### 課題2
showmap()を上のプログラムの適切な場所に加えて下さい。
### 課題3
プレーヤーが便利なように、下のようにもう少し見易く表示できるよう、showmap関数を改良して下さい。

## 上級編: 自動判定と終了判定
通常のマインスイーパーにあって、自作マインスイーパーに無い機能が見えてきたでしょうか。"0"にヒットした場合、すなわち、その場所にもその周囲8点にも爆弾が存在しない場所をクリックすると、周囲8点は安心してクリックして構わないわけです。通常のマインスイーパーだと、隣をつぎつぎに自動的に開き、ユーザーの手間を省いてくれます。

この機能はどうやって実現できるでしょうか。かなりややこしい気がしますが、実はけっこう簡単に書けます。

座標を引数とし、その周囲9点を自動的に展開する関数probe()を書いてみます。

In [ ]:
def probe(x,y,num,bombs):
    for neix in range(x-1,x+2):         #(x,y)の周囲9点について、
        for neiy in range(y-1,y+2):
            if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                bombcount = countbombs(neix,neiy,bombs)
                num[(neix,neiy)] = bombcount                

これを、bombcountが0だった時だけ呼びだします。

In [ ]:
if bombcount == 0:
    probe(x,y,num,bombs)

### 再帰による方法
0の周囲9点だけは自動で開いてくれるのですが、開いた点がまた0だった時に、その先までは開いてくれません。そこで、probe関数の中で、もしbombcountが0だった時には自分自身を呼びだして、先まで開くようにします。(再帰呼びだし)

この時注意しなければいけないことは、決して盤の外までprobeしないことです。盤の外は無限に爆弾のない広野があり、probeが終わらなくなってしまいます。

再帰しないprobeと区別するために、recursive_probeという名前をあたえました。

In [ ]:
def recursive_probe(x,y,num,bombs,mapsize):
    #(x,y)が盤の中にある場合のみ処理する。
    if 0 <= x < mapsize[0] and 0 <= y < mapsize[1]:
        for neix in range(x-1,x+2):         #(x,y)の周囲9点について、
            for neiy in range(y-1,y+2):
                if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                    bombcount = countbombs(neix,neiy,bombs)
                    num[(neix,neiy)] = bombcount
                    if bombcount == 0:      #周囲に爆弾がない場合には、その点をprobeする。
                        recursive_probe(neix,neiy,num,bombs,mapsize)

あとは、開けていない場所の数が爆弾の個数に等しくなれば、探査完了です。地図の面積はmapsize[0]\*mapsize[1]で、爆弾の総数はnbomb、探査した場所の個数はlen(num)でわかりますから、これらがある関係をみたす場合には、ループから脱出するようにして下さい。

### 待ち行列による方法
再起呼び出しは、慎重に書かないと、バグが生じた時に見付けにくい問題があります。そこで、別の方法として、待ち行列を使うことにします。

これから作業すべき点のリストを作り、前から順番に処理し、新しい作業が発生したらリストの最後に追加して、リストが空になるまで繰り返します。

In [ ]:
def probe_one(queue,num,bombs,mapsize):
    # 筆頭をとりだす
    x,y = queue.pop(0)
    #(x,y)が盤の中にある場合のみ処理する。
    if 0 <= x < mapsize[0] and 0 <= y < mapsize[1]:
        for neix in (x-1,x,x+1):         #(x,y)の周囲9点について、
            for neiy in (y-1,y,y+1):
                if (neix,neiy) not in num:  #その場所はまだ誰も調べていないなら
                    bombcount = countbombs(neix,neiy,bombs)
                    num[(neix,neiy)] = bombcount
                    if bombcount == 0:      #周囲に爆弾がない場合には、その点をqueueに追加する。
                        queue.append((neix,neiy))

人間が選んだ点のbombcountが0だった場合には、その点をqueueに追加します。

In [ ]:
if bombcount == 0:
    queue = [(x,y)]

そして、queueが空になるまでひたすらprobe_oneを呼んでqueueを消化します。

In [ ]:
    while len(queue) > 0:
        print(queue)  # queueの中身を表示させる
        probe_one(queue, num, bombs, mapsize)

これらを、プログラムのどの部分に挿しこむか考えて下さい。